In [1]:
import os.path
import pandas as pd
import numpy as np

In [2]:
def recalculate(path,year):
    aba_table_info = pd.read_csv("/Users/richadeshmukh/Desktop/TEMP/11_OralImplicitBias-master/preprocessing/audio/inputs/aba_table_info.csv")
    filedir = ""
    filedir = "/Users/richadeshmukh/Documents/Spring18/MLCS/Assignments/hw6-trees/audio_feature_2008.csv"
    speaker_dic = dict()
    #create dictionary for info table
    row_num = aba_table_info.shape[0]
    print row_num
    index = 0

    while(index<row_num):
        if(index%1000 == 0):
            print("completed "+str(index)+" rows")
        speaker_id = aba_table_info["speakerA_id"][index]
        docket_id = aba_table_info["docket_id"][index]
        docket_diction = speaker_dic.get(docket_id,-1)
        if docket_diction == -1:
            speaker_dic[docket_id] = dict()
        speaker_name = speaker_dic.get(docket_id).get(speaker_id,-1)
        if speaker_name == -1:
            speaker_dic[docket_id][speaker_id] = aba_table_info["speakerA"][index]
        index += 1

    if os.path.isfile(filedir):
        year_base_feature = pd.read_csv(filedir)
        row_num = year_base_feature.shape[0]
        #calculate mean if not exist
        if not os.path.isfile('audio_feature_'+str(year)+'_mean.csv'):
            index = 0
            vowel_id = {}
            speaker_name_dic = {}

            new_colom_speaker_name = pd.Series(np.zeros(year_base_feature.shape[0]), index=year_base_feature.index)

            # add speaker name
            while (index < row_num):
                if(index%1000 == 0):
                    print("completed "+str(index)+" rows")
                docket_id = year_base_feature["docket_id"][index]
                speaker_id = year_base_feature["speaker_id"][index]
                speaker_name = speaker_dic.get(docket_id,-1).get(speaker_id,-1)
                if speaker_name!=-1:
                    new_colom_speaker_name[index] = speaker_name
                    value = speaker_name_dic.get(speaker_name, -1)
                    if value == -1:
                        speaker_name_dic[speaker_name] = 1
                vowel = year_base_feature["vowel"][index]#added feature
                value = vowel_id.get(vowel, -1)
                if value == -1:
                    vowel_id[vowel] = 1
                index += 1
            year_base_feature["speaker_name"] = new_colom_speaker_name
            print ("Finish adding speaker_name for year: ",year)
            if not os.path.isfile('audio_feature_'+str(year)+'_withsn.csv'):
                year_base_feature.to_csv('audio_feature_'+str(year)+'_withsn.csv',header ='column_names',index = False)
            # de-mean
            # calculate group mean
            mean_df = pd.DataFrame()
            for key_1 in speaker_name_dic:
                for key_2 in vowel_id:
                    df_tmp = year_base_feature[(year_base_feature['speaker_name']==key_1)&(year_base_feature['vowel']==key_2)].mean()
                    df_tmp['speaker_name'] = key_1
                    df_tmp['vowel'] = key_2
                    mean_df = mean_df.append(df_tmp,ignore_index = True)
            mean_df.to_csv('audio_feature_'+str(year)+'_mean.csv',header='column_names',index = False)
            print ("Mean table is created.")
        else:
            mean_df = pd.read_csv('audio_feature_'+str(year)+'_mean.csv')

        year_base_feature = pd.read_csv('audio_feature_'+str(year)+'_withsn.csv')
        row_num = year_base_feature.shape[0]
        #create mean new table 
        if os.path.isfile('audio_feature_'+str(year)+'_422.csv'):
            # start from the stop point
            last_data = pd.read_csv('audio_feature_'+str(year)+'_422.csv')
            i_new = last_data.shape[0]
            print ("Start from row: ",i_new)
        else:
            i_new = 0

        tmp_data = pd.DataFrame()
        demean = lambda df: df - df.mean() 
        df_group=year_base_feature.groupby(['speaker_name','vowel']).transform(demean)  
        #print(df_group.shape)
        #print(df.shape)
        #print(df_group['f1@20%'])
        #df_group=df.groupby(['speaker_id']).mean()
        tmp_data['speaker_name']=year_base_feature['speaker_name']
        tmp_data['docket_id'] = year_base_feature['docket_id']
        tmp_data['speaker_id'] = year_base_feature["speaker_id"]

        tmp_data['text'] = year_base_feature['text']
        tmp_data['vowel'] = year_base_feature['vowel']
        tmp_data['classify'] = year_base_feature['classify']
        tmp_data['dur'] = year_base_feature['dur']
        tmp_data['t_n'] = df_group['t']

        tmp_data['f1_n'] = df_group['f1']
        tmp_data['f2_n'] = df_group['f2']
        tmp_data['f3_n'] = df_group['f3']
        tmp_data['B1_n'] = df_group['B1']
        tmp_data['B2_n'] = df_group['B2']
        tmp_data['B3_n'] = df_group['B3']

        tmp_data['f1@20%_n'] = df_group['f1@20%']
        tmp_data['f1@35%_n'] = df_group['f1@35%']
        tmp_data['f1@50%_n'] = df_group['f1@50%']
        tmp_data['f1@60%_n'] = df_group['f1@60%']
        tmp_data['f1@80%_n'] = df_group['f1@80%']
        tmp_data['f2@20%_n'] = df_group['f2@20%']
        tmp_data['f2@35%_n'] = df_group['f2@35%']
        tmp_data['f2@50%_n'] = df_group['f2@50%']
        tmp_data['f2@60%_n'] = df_group['f2@60%']
        tmp_data['f2@80%_n'] = df_group['f2@80%']

        if not os.path.isfile('audio_feature_'+str(year)+'_422.csv'):
            tmp_data.to_csv('audio_feature_'+str(year)+'_422.csv',header ='column_names',index = False)
        else:
            tmp_data.to_csv('audio_feature_'+str(year)+'_422.csv',mode = 'a',header=False,index = False)
    print ('Finish year: ',year)

In [3]:
path = "./"
recalculate(path,2008)

18512
completed 0 rows
completed 1000 rows
completed 2000 rows
completed 3000 rows
completed 4000 rows
completed 5000 rows
completed 6000 rows
completed 7000 rows
completed 8000 rows
completed 9000 rows
completed 10000 rows
completed 11000 rows
completed 12000 rows
completed 13000 rows
completed 14000 rows
completed 15000 rows
completed 16000 rows
completed 17000 rows
completed 18000 rows
completed 0 rows
completed 1000 rows
completed 2000 rows
completed 3000 rows
completed 4000 rows
completed 5000 rows
completed 6000 rows
completed 7000 rows
completed 8000 rows
completed 9000 rows
completed 10000 rows
completed 11000 rows
completed 12000 rows
completed 13000 rows
completed 14000 rows
completed 15000 rows
completed 16000 rows
completed 17000 rows
completed 18000 rows
completed 19000 rows
completed 20000 rows
completed 21000 rows
completed 22000 rows
completed 23000 rows
completed 24000 rows
completed 25000 rows
completed 26000 rows
completed 27000 rows
completed 28000 rows
completed 29

completed 359000 rows
completed 360000 rows
completed 361000 rows
completed 362000 rows
completed 363000 rows
completed 364000 rows
completed 365000 rows
completed 366000 rows
completed 367000 rows
completed 368000 rows
completed 369000 rows
completed 370000 rows
completed 371000 rows
completed 372000 rows
completed 373000 rows
completed 374000 rows
completed 375000 rows
completed 376000 rows
completed 377000 rows
completed 378000 rows
completed 379000 rows
completed 380000 rows
completed 381000 rows
completed 382000 rows
completed 383000 rows
completed 384000 rows
completed 385000 rows
completed 386000 rows
completed 387000 rows
completed 388000 rows
completed 389000 rows
completed 390000 rows
completed 391000 rows
completed 392000 rows
completed 393000 rows
completed 394000 rows
completed 395000 rows
completed 396000 rows
completed 397000 rows
completed 398000 rows
completed 399000 rows
completed 400000 rows
completed 401000 rows
completed 402000 rows
completed 403000 rows
completed 

completed 732000 rows
completed 733000 rows
completed 734000 rows
completed 735000 rows
completed 736000 rows
completed 737000 rows
completed 738000 rows
completed 739000 rows
completed 740000 rows
completed 741000 rows
completed 742000 rows
completed 743000 rows
completed 744000 rows
completed 745000 rows
completed 746000 rows
completed 747000 rows
completed 748000 rows
completed 749000 rows
completed 750000 rows
completed 751000 rows
completed 752000 rows
completed 753000 rows
completed 754000 rows
completed 755000 rows
completed 756000 rows
completed 757000 rows
completed 758000 rows
completed 759000 rows
completed 760000 rows
completed 761000 rows
completed 762000 rows
completed 763000 rows
completed 764000 rows
completed 765000 rows
completed 766000 rows
completed 767000 rows
completed 768000 rows
completed 769000 rows
completed 770000 rows
completed 771000 rows
completed 772000 rows
completed 773000 rows
completed 774000 rows
completed 775000 rows
completed 776000 rows
completed 